# **Activity 1: Energy-Minimized PHA oligomer**

Before you proceed this workflow, I encourage you to follow the tutorials on ```00_tutorials/``` folder prior programming to have everything downloaded.

In this workflow, we will retrieve energy-minimized PHA oligomer 3D structure for molecular docking and dynamics.

Before you continue, please make sure you have already followed the instructions from ```00_tutorials/04_setting_vscode.md```.

Make sure you have already installed ```RDKit``` and ```Openbabel``` libraries. Otherwise, copy the following lines and paste it on the terminal.

1. ```conda activate vina```
3. ```sudo apt install openbabel```
2. ```conda install openbabel rdkit```


## **1. Import libraries**

In [21]:
# Library for compound manipulation
from rdkit import Chem
from rdkit.Chem import AllChem, rdChemReactions
# Library for energy minimization
from openbabel import pybel
# Library for file handling and export
import os

## **2. Define monomer and polymerization reaction**

Get the SMILES of monomer from PubChem: https://pubchem.ncbi.nlm.nih.gov/

In [22]:
# Define monomer
monomer_smiles = "C[C@H](CC(=O)O)O"  # PHB monomer
# Convert monomer to RdKit-readable
monomer = Chem.MolFromSmiles(monomer_smiles)

print(f"Monomer SMILES: {monomer_smiles}")
print(f"Monomer formula: {Chem.rdMolDescriptors.CalcMolFormula(monomer)}")
print(f"Monomer MW: {Chem.rdMolDescriptors.CalcExactMolWt(monomer):2f}")

Monomer SMILES: C[C@H](CC(=O)O)O
Monomer formula: C4H8O3
Monomer MW: 104.047344


In [23]:
# Define polymer reaction (SMART structure)
# [Acid-C:1] + [Alcohol-O:4] -> [Ester]
rxn_smarts = '[C:1](=[O:2])[OH].[C:3][OH:4]>>[C:1](=[O:2])[O:4][C:3]'
# Convert polymer reaction into RdKit-readable
rxn = rdChemReactions.ReactionFromSmarts(rxn_smarts)

print(f"Reaction SMARTS: {rxn_smarts}")
print("Reaction: Carboxylic acid + Alcohol -> Ester\n")

Reaction SMARTS: [C:1](=[O:2])[OH].[C:3][OH:4]>>[C:1](=[O:2])[O:4][C:3]
Reaction: Carboxylic acid + Alcohol -> Ester



## **3. Build the oligomer(s) using a user-customed function**

In [24]:
def build_phb_oligomer(num_units:int):
    """
    Build a PHB oligomer with a specified number of units
    
    Parameters:
    num_units (int): Number of PHB units in the oligomer
    
    Returns:
    oligomer (RDKit Mol): Final oligomer
    """
    if num_units < 1:
        raise ValueError("Number of units must be at least 1")
    
    # Start with the monomer
    oligomer = Chem.Mol(monomer)
    Chem.SanitizeMol(oligomer)
    
    # Iteratively add units
    for i in range(num_units - 1):
        products = rxn.RunReactants((oligomer, monomer))
        if products:
            oligomer = products[0][0]
            Chem.SanitizeMol(oligomer)
            #print(f"After reaction {i+1}: {Chem.MolToSmiles(oligomer)}")
        else:
            print(f"Warning: No reaction occurred at iteration {i+1}")
            return None

    Chem.SanitizeMol(oligomer)
    print(f"Oligomer SMILES: {Chem.MolToSmiles(oligomer)}")
    print(f"Number of Carbon atoms: {len(oligomer.GetSubstructMatches(Chem.MolFromSmarts('C')))}")
    return oligomer

## **4. Generate oligomer 3D structure(s)**

In [25]:
oligomers_3d = {} # Dictionary to store 3D structures

for num_units in range(1, 13):  # 1 to 12 units; Modify this range as needed
    print(f"\nBuilding {num_units}-unit PHB oligomer...")
    
    # Build oligomer
    oligomer = build_phb_oligomer(num_units)
    
    if oligomer is None:
        print(f"\t- Failed to build {num_units}-unit oligomer")
        continue
    
    # Add hydrogens
    oligomer_h = Chem.AddHs(oligomer)
    
    # Generate 3D coordinates
    AllChem.EmbedMolecule(oligomer_h, AllChem.ETKDG())
    
    # Quick MMFF94 optimization before OpenBabel
    AllChem.MMFFOptimizeMolecule(oligomer_h)

    # Store 3D structure in Dictionary
    oligomers_3d[num_units] = oligomer_h

print("\n3D conversion completed.")


Building 1-unit PHB oligomer...
Oligomer SMILES: C[C@@H](O)CC(=O)O
Number of Carbon atoms: 4

Building 2-unit PHB oligomer...
Oligomer SMILES: C[C@H](CC(=O)O)OC(=O)C[C@@H](C)O
Number of Carbon atoms: 8

Building 3-unit PHB oligomer...
Oligomer SMILES: C[C@H](CC(=O)O)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)O
Number of Carbon atoms: 12

Building 4-unit PHB oligomer...
Oligomer SMILES: C[C@H](CC(=O)O)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)O
Number of Carbon atoms: 16

Building 5-unit PHB oligomer...
Oligomer SMILES: C[C@H](CC(=O)O)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)O
Number of Carbon atoms: 20

Building 6-unit PHB oligomer...
Oligomer SMILES: C[C@H](CC(=O)O)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)O
Number of Carbon atoms: 24

Building 7-unit PHB oligomer...
Oligomer SMILES: C[C@H](CC(=O)O)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)OC(=O)C[C@@H](C)O
Number of Carbon atoms: 28

Buildi

## **5. Energy minimize structure(s) using OpenBabel**

In [26]:
# Create output directory if it doesn't exist
output_dir = "files/"
os.makedirs(output_dir, exist_ok=True)

minimized_molecules = {} # Dictionary to store energy-minimized structures

for num_units in range(1, 13): # Modify this range as needed
    if num_units not in oligomers_3d:
        print(f"\nSkipping {num_units}-unit (not generated)")
        continue
    
    print(f"\nMinimizing {num_units}-unit oligomer...")
    
    # Convert RDKit molecule to MOL block
    oligomer_3d = oligomers_3d[num_units]
    mol_block = Chem.MolToMolBlock(oligomer_3d)
    
    # Read into OpenBabel
    ob_mol = pybel.readstring("mol", mol_block)
    
    # Perform energy minimization
    ob_mol.localopt(forcefield='mmff94', steps=500)

    minimized_molecules[num_units] = ob_mol

print("\nEnergy minimization completed.")


Minimizing 1-unit oligomer...

Minimizing 2-unit oligomer...

Minimizing 3-unit oligomer...

Minimizing 4-unit oligomer...

Minimizing 5-unit oligomer...

Minimizing 6-unit oligomer...

Minimizing 7-unit oligomer...

Minimizing 8-unit oligomer...

Minimizing 9-unit oligomer...

Minimizing 10-unit oligomer...

Minimizing 11-unit oligomer...

Minimizing 12-unit oligomer...

Energy minimization completed.


## **6. Export to PDB**

In [27]:
for num_units in range(1, 13): # Modify this range as needed
    if num_units not in minimized_molecules:
        print(f"\nSkipping {num_units}-unit (not minimized)")
        continue
    
    ob_mol = minimized_molecules[num_units]
    
    # Create filename
    output_file = os.path.join(output_dir, f"phb_{num_units}_unit.pdb")
    
    # Write PDB file
    ob_mol.write("pdb", output_file, overwrite=True)

## END